<a href="https://colab.research.google.com/github/King-Rian/Project-3/blob/main/project_3_Lou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install emoji library
!pip install emoji
!pip install torch
from google.colab import files
import zipfile
import json
import emoji
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 11.8 MB/s eta 0:00:00


In [5]:
# Retrieve the 'emojis.json' zippped file

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Extract the ZIP file
with zipfile.ZipFile('emojis.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Step 3: Load the extracted JSON file
emoji_file = 'emojis.json'

# Using pandas
emoji_df = pd.read_json(emoji_file)
print("Dataset loaded successfully. Sample data:")
print(emoji_df.head())


Saving emojis.json.zip to emojis.json.zip
Dataset loaded successfully. Sample data:
                                            category  \
0  Miscellaneous Symbols And Pictographs -> Emoji...   
1  Miscellaneous Symbols And Pictographs -> Emoji...   
2                                               None   
3  Miscellaneous Symbols And Pictographs -> Emoji...   
4                                               None   

                                            keywords  \
0  [dark skin tone, hand, forbidden, gesture, wom...   
1                     [dark skin tone, woman, guard]   
2                 [racing, running, woman, marathon]   
3  [gymnastics, medium-light skin tone, woman, ca...   
4                                      [woman, golf]   

                                          definition  \
0  The Woman Gesturing Not OK, Type-6 emoji is a ...   
1  The Female Guard, Type-6 emoji is a sequence o...   
2  The female version of the ?? Runner emoji. The...   
3  The Woman Doing

In [6]:
# Check the shape, features, and datatypes of the df
emoji_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    1988 non-null   object
 1   keywords    2389 non-null   object
 2   definition  2389 non-null   object
 3   unicode     2389 non-null   object
 4   name        2389 non-null   object
 5   shortcode   845 non-null    object
 6   senses      2389 non-null   object
dtypes: object(7)
memory usage: 130.8+ KB


In [7]:
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

# Create a new emojis DataFrame with 'name' and 'unicode'
key_value_pairs_df = emoji_df[['name', 'unicode']]

# Process the 'name' column to include : at the beginning and end of the emoji name
key_value_pairs_df['name'] = key_value_pairs_df['name'].apply(
    lambda x: f":{x.split(':')[0].strip()}:"  # Remove everything after the first colon and add colons
)

# Convert the unicode column to emoji symbols
key_value_pairs_df['emoji'] = key_value_pairs_df['name'].apply(emoji.emojize)

# Filter out rows where 'unicode' contains more than one Unicode value
# We check for the number of spaces and ensure it is less than 1
# Eliminating complex emojis and ZWJ sequence values
filtered_df = key_value_pairs_df[key_value_pairs_df['unicode'].str.count(' ') < 1]

# Remove rows where 'emoji' contains text with a beginning and ending ':' - which means that emoji was not found for that :name:
filtered_df = key_value_pairs_df[~key_value_pairs_df['emoji'].str.match(r'^:.*:$')]

# Create a new df and rename features to read for concatination with the next data set
new_filtered_df = filtered_df[['name', 'emoji']]
new_filtered_df = new_filtered_df.rename(columns={'name': 'label'})

# Display the DataFrame with emoji symbols
new_filtered_df.head()

<ipython-input-7-5759284d03f7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_value_pairs_df['name'] = key_value_pairs_df['name'].apply(
<ipython-input-7-5759284d03f7>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_value_pairs_df['emoji'] = key_value_pairs_df['name'].apply(emoji.emojize)


,label,emoji
15,:squid:,🦑
16,:shrimp:,🦐
17,:rhinoceros:,🦏
18,:lizard:,🦎
19,:gorilla:,🦍


In [8]:
# Create an emoji dictionary with labels and their Unicode values from https://carpedm20.github.io/emoji/docs/index.html
emoji_dict = {emoji.demojize(e): e for e in emoji.EMOJI_DATA}

# Show how many key value pairs we retrieved from emoji library
records = len(emoji_dict)
print(f"Number of records: {records}")

# Create new df for use in concatenation the two data frames
emoji_dictionary_df = pd.DataFrame(emoji_dict.items(), columns=['label', 'emoji'])
emoji_dictionary_df.head()
#emoji_dictionary_df.info()

Number of records: 3790


,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [9]:
# Concatentate both dataframes / sources and drop the index
combined_emoji_df = pd.concat([emoji_dictionary_df, new_filtered_df], ignore_index=True)

# Show the key pair data frame with training data
combined_emoji_df.head()

,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [10]:
# Step 1: Prepare the Data
# Create a mapping of emojis to indices
emoji_to_index = {emoji: idx for idx, emoji in enumerate(combined_emoji_df['emoji'].unique())}
index_to_emoji = {idx: emoji for emoji, idx in emoji_to_index.items()}

In [11]:
# Step 2: Text Encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(phrases):
    encoded = tokenizer(phrases.tolist(), padding=True, truncation=True, return_tensors='pt')
    return encoded['input_ids'], encoded['attention_mask']

input_ids, attention_masks = tokenize_data(combined_emoji_df['label'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
# Step 3: Prepare Emoji Sequences to Indices
combined_emoji_df['emoji_index'] = combined_emoji_df['emoji'].map(emoji_to_index)
target_indices = torch.tensor(combined_emoji_df['emoji_index'].tolist())  # Convert to tensor


In [13]:
# Step 4: Create DataLoader
dataset = TensorDataset(input_ids, attention_masks, target_indices)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)


In [14]:
# Step 5: Build the Model
class BertEmojiClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BertEmojiClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  # Get the pooled output (CLS token)
        pooled_output = self.dropout(pooled_output)
        return self.fc(pooled_output)

# Initialize the model
num_classes = len(emoji_to_index)
model = BertEmojiClassifier(num_classes)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [15]:
# Step 6: Training the Model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_masks, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_masks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

Epoch 1/3, Loss: 8.279444855124204
Epoch 2/3, Loss: 8.196561931102751
Epoch 3/3, Loss: 8.150538746898302


In [16]:
# Step 7: Evaluate the Model
model.eval()
val_accuracy = 0
total = 0

with torch.no_grad():
    for batch in train_loader:
        input_ids, attention_masks, labels = [b.to(device) for b in batch]
        outputs = model(input_ids, attention_masks)

        _, predicted = torch.max(outputs, 1)
        val_accuracy += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {val_accuracy / total:.4f}")

Validation Accuracy: 0.0191


In [17]:
# Step 8: Make Predictions
def predict_emoji(phrase):
    model.eval()
    inputs = tokenizer(phrase, return_tensors='pt', padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = model(inputs['input_ids'], inputs['attention_mask'])
        _, predicted_index = torch.max(outputs, 1)

    return index_to_emoji[predicted_index.item()]

In [20]:
# Step 9: Test Predictions
new_phrase = "happy"
predicted_emoji = predict_emoji(new_phrase)
print(f"The predicted emoji for '{new_phrase}' is: {predicted_emoji}")

The predicted emoji for 'happy' is: 👪


In [ ]:
# Step 10: Build out a user interface that accepts text entry and uses the model to predict output
